# Pipeline de Análisis de Outliers

In [ ]:
# Tratamiento de datos
# ==============================================================================
import numpy as np
import pandas as pd

# Graficos
# ==============================================================================
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns; sns.set()
# sns.set_style('darkgrid', {'axes.facecolor': '.9'})
# %matplotlib inline

# Preprocesado y modelado
# ==============================================================================
# from sklearn.preprocessing import StandardScaler
import pickle
import os
from scipy.stats.mstats import winsorize
import statistics
from datetime import date

from sklearn.ensemble import IsolationForest
from sklearn.covariance import EllipticEnvelope
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM

# Configuracion warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

# Semilla de aleatorizacion
# ==============================================================================
SEED = 666

In [ ]:
#FUNCIONES PARA ENTRENAR LOS DISTINTOS MODELOS DE DETECCION
def EntrenarIF(datos, ruta_modelo):
    """
    Entrenar un modelo Isolation Forest y guardarlo en la direccion dada.
    
    datos (pandas.df) = datos de entrenamiento
    ruta_modelo (str) = nombre del fichero en el que se guardara el modelo. Debe acabar en .sav
    """
    #Definicion y entrenamiento del iforest
    modelo_iforest = IsolationForest(n_estimators = 1000,
                                     max_samples = 0.75,
                                     contamination = 0.01,
                                     n_jobs = -1,
                                     random_state = SEED)
    modelo_iforest = modelo_iforest.fit(X = datos)
    #Guardar el modelo entrenado
    pickle.dump(modelo_iforest, open(ruta_modelo, "wb"))
    
    return modelo_iforest


def EntrenarMCD(datos, ruta_modelo):
    """
    Entrenar un modelo MCD y guardarlo en la direccion dada.
    
    datos (pandas.df) = datos de entrenamiento
    ruta_modelo (str) = nombre del fichero en el que se guardara el modelo. Debe acabar en .sav
    """
    #Definicion y entrenamiento del iforest
    modelo_mcd = EllipticEnvelope(contamination = 0.01,
                                  random_state = SEED)
    modelo_mcd = modelo_mcd.fit(X = datos)
    #Guardar el modelo entrenado
    pickle.dump(modelo_mcd, open(ruta_modelo, "wb"))
    
    return modelo_mcd


def EntrenarLOF(datos, ruta_modelo):
    """
    Entrenar un modelo LOF y guardarlo en la direccion dada.
    
    datos (pandas.df) = datos de entrenamiento
    ruta_modelo (str) = nombre del fichero en el que se guardara el modelo. Debe acabar en .sav
    """
    #Definicion y entrenamiento del iforest
    modelo_lof = LocalOutlierFactor(novelty=True)
    modelo_lof = modelo_lof.fit(X = datos)
    #Guardar el modelo entrenado
    pickle.dump(modelo_lof, open(ruta_modelo, "wb"))
    
    return modelo_lof


def EntrenarSVM(datos, ruta_modelo):
    """
    Entrenar un modelo SVM y guardarlo en la direccion dada.
    
    datos (pandas.df) = datos de entrenamiento
    ruta_modelo (str) = nombre del fichero en el que se guardara el modelo. Debe acabar en .sav
    """
    #Definicion y entrenamiento del iforest
    modelo_svm = OneClassSVM(nu=0.01)
    modelo_svm = modelo_svm.fit(X = datos)
    #Guardar el modelo entrenado
    pickle.dump(modelo_svm, open(ruta_modelo, "wb"))
    
    return modelo_svm

In [ ]:
#FUNCIONES AUXILIARES PARA DETERMINAR OUTLIERS EN FUNCION DEL IQR Y VISUALIZAR SCORES
def CrearDirectorio(path):
    """
    Comprobar si un directorio existe y, si no es asi, crearlo.
    
    path (str) = directorio que se quiere crear
    """
    dir_existe = os.path.exists(path)
    if not dir_existe:
        os.makedirs(path)
    return


def ValorarScoresIQR(scores, verbose = True):
    """
    Funcion para determinar a partir de que score de una TDA se declara un dato como anomalo
    usando el rango intercuartilico (IQR).
    Devuelve una lista de bool que dice si los datos son anomalos (True) y el umbral para la score.
    """
    #Obtener el IQR
    Q1 = np.percentile(scores, 25)
    Q3 = np.percentile(scores, 75)
    IQR = Q3 - Q1

    scores = scores.tolist()
    #Umbral inferior e indices
    umbral = Q1 - 3 * IQR
    outs = list(scores <= umbral)

    if verbose:
        print("- Sobrepasan el umbral:", sum(outs))
    
    return outs, umbral


def HistUmbrales(scores, umbral):
    """
    Funcion para mostrar el histograma de las scores de un modelo de deteccion de anomalias
    junto con los umbrales inferior y superior.
    
    scores (list) = puntuacion de anomalia de los puntos del dataset
    umbral_sup (float) = valor a partir del cual una observacion es considerada anomalia
    umbral_inf (float) = valor por debajo del cual una observacion es considerada anomalia
    """
    fig, ax = plt.subplots(figsize=(7, 3.5))
    sns.distplot(scores, hist=True, kde=True, 
                 color = 'blue', 
                 hist_kws = {'edgecolor':'black'},
                 kde_kws = {'linewidth': 2},
                 ax = ax
                )
    ax.axvline(umbral, c='red', linestyle='--') #umbral
    plt.show()


def EvaluarModelo(modelo, datos, verbose = True, visualizar = True):
    """
    Generar scores de anomalia con un modelo entrenado y evaluarlas. Los modelos
    pueden ser de Isolation Forest, MCD, LOF o SVM.
    
    modelo = modelo entrenado
    datos = datos sobre los que realizar la evaluacion
    """
    #Obtener las scores de las muestras y los outliers
    scores = modelo.score_samples(X = datos)
    idx_out, umbral = ValorarScoresIQR(scores, verbose)
    
    if visualizar:
        HistUmbrales(scores, umbral)
    
    return scores, idx_out, umbral

In [ ]:
#FUNCION PARA ENTRENAR LAS TDA ELEGIDAS
def EntrenarTDA(datos, rutas_modelos):
    """
    Entrenar todas las TDA, calcular las scores, obtener umbrales a partir de los cuales
    se determine como outlier a cada observacion, guardar los datos declarados como outlier
    por cada tecnica separados y obtener los indices de los mismos.
    """
    #Comprobar si directorio modelos existe
    CrearDirectorio(path = "./modelos/")
    #Entrenar modelos de todas las TDAs
    modelo_if = EntrenarIF(datos, rutas_modelos[0])
    modelo_mcd = EntrenarMCD(datos, rutas_modelos[1])
    modelo_lof = EntrenarLOF(datos, rutas_modelos[2])
    modelo_svm = EntrenarSVM(datos, rutas_modelos[3])
    
    #Obtener scores modelos y visualizar si se quiere
    scores_if, idx_out_if, umbral_if = EvaluarModelo(modelo_if, datos)
    scores_mcd, idx_out_mcd, umbral_mcd = EvaluarModelo(modelo_mcd, datos)
    scores_lof, idx_out_lof, umbral_lof = EvaluarModelo(modelo_lof, datos)
    scores_svm, idx_out_svm, umbral_svm = EvaluarModelo(modelo_svm, datos)
    
    #Añadir puntuaciones de los distintos metodos
    datos["scores_if"] = scores_if
    datos["scores_mcd"] = scores_mcd
    datos["scores_lof"] = scores_lof
    datos["scores_svm"] = scores_svm
    
    #Comprobar que el directorio para outliers existe
    CrearDirectorio(path = "./outliers/")
    #Obtener muestras outliers y guardarlas separadas
    lista_idx = [idx_out_if, idx_out_mcd, idx_out_lof, idx_out_svm]
    lista_outs = list()
    for idx in lista_idx:
        lista_outs.append(datos.iloc[idx,])
    #Guardar cada conjunto de outliers con un nombre distinto
    nom_modelos = ["iforest", "mcd", "lof", "svm"]
    for i in range(len(nom_modelos)):
        direccion = "./outliers/datos_" + nom_modelos[i] + ".xlsx"
        lista_outs[i].to_excel(direccion)
    
    return lista_idx, datos

## Análisis de Outliers Unidimensionales

In [ ]:
#Filtrar primero outliers por variable
outs, umbral = ValorarScoresIQR(df[variable])
#Guardamos los datos sin outliers por importe
bool_mask = [not elem for elem in outs]
df_clean = df.iloc[bool_mask]

print("Tamaño actual de los datos:", df_clean.shape)

## Análisis de Outliers con TDA

In [ ]:
rutas_modelos = ["./modelos/modelo_ifo_datos.sav", "./modelos/modelo_mcd_datos.sav",
                 "./modelos/modelo_lof_datos.sav", "./modelos/modelo_svm_datos.sav"]
lista_idx, df_scores = EntrenarTDA(df_clean, rutas_modelos)

In [ ]:
#Crear columnas de ordenación por método
cols_scores = ["scores_if", "scores_mcd", "scores_lof", "scores_svm"]
cols_orden = ["orden_if", "orden_mcd", "orden_lof", "orden_svm"]
for i in range(len(cols_scores)):
    df_scores = df_scores.sort_values(cols_scores[i])
    df_scores[cols_orden[i]] = range(1, len(df_scores) + 1)

#Crear columna que combine el orden de outliers para todas las TDA
df_scores["orden_global"] = [0] * df_scores.shape[0] #columna de 0
for col in cols_orden:
    df_scores["orden_global"] = df_scores["orden_global"] + df_scores[col]

df_scores = df_scores.sort_values("ID")
#Unir a los datos originales todas las columnas obtenidas
df_final = pd.merge(df, df_scores, on=["ID"])

In [ ]:
#Exportar resultados
df_final.to_excel("../data/datos_anomalias.xlsx", index = True) #index True si contiene el id como indice